In [5]:
pip install ollama

In [8]:
import ollama
from IPython.display import Markdown, display

response1 = ollama.chat(
    model='llama3.2',
    messages = [{"role":"user",
                 "content": "Who wrote the book Lord of the Rings?"
                 }])
display(Markdown(response1.message.content))

The book "Lord of the Rings" was written by J.R.R. Tolkien (John Ronald Reuel Tolkien).

In [12]:
pip install distilabel

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tblib

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

from distilabel.models import OllamaLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub
from distilabel.steps.tasks import TextGeneration, UltraFeedback
from distilabel.steps import GroupColumns

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import pandas as pd                                                                          │
│   2                                                                                              │
│   3 from distilabel.models import OllamaLLM                                                      │
│ ❱ 4 from distilabel.pipeline import Pipeline                                                     │
│   5 from distilabel.steps import LoadDataFromHub                                                 │
│   6 from distilabel.steps.tasks import TextGeneration, UltraFeedback                             │
│   7 from distilabel.steps import GroupColumns                                                    │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │        exit = <IPython.core.autocall.ZMQExitAutocall object at 0x000001785C44A5D0>           │ │
│ │ get_ipython = <bound method InteractiveShell.get_ipython of                                  │ │
│ │               <ipykernel.zmqshell.ZMQInteractiveShell object at 0x000001785C4360D0>>         │ │
│ │          In = [                                                                              │ │
│ │               │   '',                                                                        │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+200,                                                                │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+200,                                                                │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+200,                                                                │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+200,                                                                │ │
│ │               │   "get_ipython().run_line_magic('pip', 'install ollama')",                   │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+200,                                                                │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+198,                                                                │ │
│ │               │   'import ollama\nfrom IPython.display import Markdown, display\n\nresponse1 │ │
│ │               = ollama.'+193,                                                                │ │
│ │               │   'import pandas as pd\n\nfrom distilabel.llms import OllamaLLM\nfrom        │ │
│ │               distilabel.pipel'+171,                                                         │ │
│ │               │   ... +7                                                                     │ │
│ │               ]                                                                              │ │
│ │      ollama = <module 'ollama' from                                                          │ │
│ │               'c:\\Users\\adria\\anaconda3\\Lib\\site-packages\\ollama\\__init__.py'>        │ │
│ │         Out = {}                                                                             │ │
│ │          pd = <module 'pandas' from                      

In [ ]:
import ollama
import pandas as pd
from IPython.display import display, Markdown

# Define the topic for tweet generation
topic = "artificial intelligence in education"

# Number of tweets to generate
n_tweets = 30

# Prompt to send to the model
prompt = f"""
Generate {n_tweets} short and synthetic tweets in English about the topic "{topic}". 
Each tweet should be no more than 280 characters and should sound like it was written by a real person.
Do not number the tweets. Just separate them with line breaks.
"""

# Send request to the model
response = ollama.chat(
    model='llama3.2-vision',
    messages = [{"role": "user", "content": prompt}]
)

# Extract and process tweets
tweets_text = response['message']['content']
tweets_list = [tweet.strip() for tweet in tweets_text.split('\n') if tweet.strip()]

# Create DataFrame from the tweet list
df_tweets = pd.DataFrame({'tweet': tweets_list})

# Display the first few tweets
display(Markdown(f"### {len(df_tweets)} Tweets Generated on: *{topic}*"))
display(df_tweets.head())

### 28 Tweets Generated on: *artificial intelligence in education*

,tweet
0,AI in ed is the future! Personalized learning ...
1,"Teachers, don't be afraid of AI taking over yo..."
2,Did you know that AI can help detect student c...
3,Personalized learning with AI means no more on...
4,The future of education is bright indeed! AI w...


In [18]:
df_tweets.to_csv("synthetic_tweets.csv", index=False)

In [ ]:
import ollama
import pandas as pd
from IPython.display import display, Markdown

# Lista de temas para geração de tweets
topics = [
    "Quotes in the style of Socrates on Twitter",
    "Parodies of motivational phrases",
    "Intentionally cringe-style tweets",
    "Funny confessions",
    "Deepfakes and synthetic reality",
    "Ethical dilemmas in AI",
    "Productivity culture",
    "Procrastination in college",
    "Existential quotes",
    "Freedom and technology",
    "Anxiety before exams",
    "Tweets as if written by fictional characters",
    "Internet slang",
    "Crushes and flirting",
    "Fictional relationship arguments (DRs)",
    "Quantum physics in 280 characters",
    "Monday-style tweets",
    "Absurd but real situations",
    "Unsent messages",
    "Motivational quotes for studying"
]

# Número de tweets por tema
n_tweets = 5000

# Dicionário para armazenar os DataFrames por tema
all_tweets = {}

# Loop para gerar tweets para cada tema
for topic in topics:
    prompt = f"""
    Generate {n_tweets} short and synthetic tweets in English about the topic "{topic}". 
    Each tweet should be no more than 280 characters and should sound like it was written by a real person. 
    Do not number the tweets. Just separate them with line breaks.
    """
    
    # Envia o prompt ao modelo
    response = ollama.chat(
        model='llama3.2-vision', #alterar aqui de acordo com a IA que for utilizada
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Processa os tweets retornados
    tweets_text = response['message']['content']
    tweets_list = [tweet.strip() for tweet in tweets_text.split('\n') if tweet.strip()]
    
    # Cria um DataFrame com os tweets
    df_tweets = pd.DataFrame({'tweet': tweets_list})
    
    # Armazena o DataFrame no dicionário
    all_tweets[topic] = df_tweets

    # Salva em arquivo CSV (opcional)
    filename = topic.lower().replace(" ", "_").replace("/", "_") + ".csv"
    df_tweets.to_csv(filename, index=False)
